---

# Atenção!

Lembre-se de clonar este notebook antes de tentar editar as células de código.

Para isso, basta seguir os passos:

File -> Save a copy in Drive



---

In [ ]:
!git clone https://github.com/loggi/loggibud
%cd /content/loggibud/

# Instale as dependências do projeto
!pip install poetry
!poetry install
# Se você estiver executando esse script localmente, não precisa dos dois comandos abaixo
!poetry export -f requirements.txt --without-hashes --output requirements.txt
!pip install -r requirements.txt

# Verifique se tudo funcionou executando os testes
!poetry run pytest -s -v tests/

# Baixe os dados compilados
!wget -nc https://loggibud.s3.amazonaws.com/dataset.zip
!unzip -n dataset.zip

# Verifique que a pasta `data/` agora não está mais vazia
!ls data/

Aqui está também o código do nosso solver completo:

In [2]:
from ortools.constraint_solver import pywrapcp

from loggibud.v1.distances import calculate_distance_matrix_m, OSRMConfig
from loggibud.v1.types import CVRPSolution, CVRPSolutionVehicle, CVRPInstance


def solve_loggibud_vrp(problem):
    distance_matrix = _compute_distance_matrix(problem)
    node_demands = _compute_node_demands(problem)
    vehicle_capacity = problem.vehicle_capacity

    # Chama o solver de antes
    routes, distance = solve_vrp_ortools2(
        distance_matrix, node_demands, vehicle_capacity
    )

    # Cria uma solução com o formato `CVRPSolution`
    return _create_cvrp_solution(problem, routes)

def _compute_distance_matrix(problem):
    config = OSRMConfig(host="http://localhost:5000")

    points = [problem.origin]
    for delivery in problem.deliveries:
        points.append(delivery.point)

    return calculate_distance_matrix_m(points, config)


def _compute_node_demands(problem):
    """Retorna uma lista com as demandas de cada nó"""
    node_demands = [0]  # inicializa com a demanda nula da origem
    for delivery in problem.deliveries:
        node_demands.append(delivery.size)

    return node_demands

def _create_cvrp_solution(problem, routes):
    vehicles = []
    for route in routes:
        vehicle = _create_cvrp_vehicle(problem, route)
        vehicles.append(vehicle)

    # Com os veículos, construímos o objeto `CVRPSolution`
    return CVRPSolution(
        name=problem.name,
        vehicles=vehicles
    )

def _create_cvrp_vehicle(problem, route):
    """
    Constrói um objeto do tipo `CVRPSolutionVehicle` a partir de uma rota
    """
    deliveries = []
    for node in route[1:-1]:
        deliveries.append(problem.deliveries[node - 1])

    return CVRPSolutionVehicle(origin=problem.origin, deliveries=deliveries)


def solve_vrp_ortools2(
    distance_matrix, node_demands, vehicle_capacity
):
    n = distance_matrix.shape[0]  # número de nós do problema
    depot_node = 0  # número do nó que representa o ponto de origem

    # Vamos usar `n` como número de veículos, pois haveria na pior das hipóteses
    # um veículo entregando cada pacote
    num_vehicles = n
    manager = pywrapcp.RoutingIndexManager(n, num_vehicles, depot_node)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(i, j):
        # `i` e `j` são índices internos do OR-Tools. Precisamos primeiro
        # convertê-los em nós do nosso problema
        ni = manager.IndexToNode(i)
        nj = manager.IndexToNode(j)
        return distance_matrix[ni, nj]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Adiciona a restrição de capacidade
    def demand_callback(from_index):
        """Retorna a demanda de um nó"""
        from_node = manager.IndexToNode(from_index)
        return node_demands[from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback
    )
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        [vehicle_capacity] * num_vehicles,
        True,  # start cumul to zero
        'Capacity'
    )

    # Resolve o problema com métodos default
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    solution = routing.SolveWithParameters(search_parameters)

    # Caso não haja solução factível, retorne uma lista vazia como rotas e o
    # valor -1 como distância total
    if not solution:
        return [], -1

    # Constrói as rotas finais
    def create_vehicle_route(vehicle_index):
        route = []
        index = routing.Start(vehicle_index)
        node = manager.IndexToNode(index)
        route.append(node)

        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            node = manager.IndexToNode(index)
            route.append(node)
        return route

    routes = []
    for vehicle_index in range(num_vehicles):
        # Adicione apenas as rotas com mais que apenas [0, 0], ou seja, apenas
        # aquelas com ao menos três pontos
        route = create_vehicle_route(vehicle_index)
        if len(route) > 2:
            routes.append(route)

    return routes, solution.ObjectiveValue()

# Exercício 1

## `dev/df-0/cvrp-0-df-90.json`

In [5]:
# Comece carregando a instância
from loggibud.v1.types import CVRPInstance


file_path = "../data/cvrp-instances-1.0/dev/df-0/cvrp-0-df-90.json"
problem = CVRPInstance.from_file(file_path)
problem

CVRPInstance(name='cvrp-0-df-90', region='df-0', origin=Point(lng=-47.802664728268745, lat=-15.657013854445248), vehicle_capacity=180, deliveries=[Delivery(id='50a13dc4e175645e22d7eac9c9ee35c5', point=Point(lng=-47.75847657230832, lat=-15.652541341978136), size=6), Delivery(id='e98f946594f5939271c3b82e439ff71c', point=Point(lng=-47.756701028749525, lat=-15.65232270602603), size=1), Delivery(id='c0b320558716e0cf15ce45d8b8189046', point=Point(lng=-47.7609277270516, lat=-15.65246980577859), size=7), Delivery(id='d1dd282d0b84b27e945f016c08447d2a', point=Point(lng=-47.75623444551221, lat=-15.659632429949554), size=10), Delivery(id='52dd17bd1b75c883a4b665e96d95e3d', point=Point(lng=-47.74744591667285, lat=-15.701436474802009), size=3), Delivery(id='79aa37298cd61e575191af560cd93650', point=Point(lng=-47.72718704525455, lat=-15.628607686615675), size=4), Delivery(id='9742cf900fa7927783690026d3ab797d', point=Point(lng=-47.73346092903645, lat=-15.632251022011443), size=3), Delivery(id='a8d59afb3

In [6]:
print(f"A capacidade de cada veículo é: {problem.vehicle_capacity}")
print(f"A localização do ponto de partida é: {problem.origin}")
print(f"Esta instância possui um total de {len(problem.deliveries)} entregas")

A capacidade de cada veículo é: 180
A localização do ponto de partida é: Point(lng=-47.802664728268745, lat=-15.657013854445248)
Esta instância possui um total de 986 entregas


In [7]:
# Resolve o problema (pode levar um tempo)
solution = solve_loggibud_vrp(problem)
solution

CVRPSolution(name='cvrp-0-df-90', vehicles=[CVRPSolutionVehicle(origin=Point(lng=-47.802664728268745, lat=-15.657013854445248), deliveries=[Delivery(id='485df7d158f4be72db0e838933c617dc', point=Point(lng=-47.70375767069516, lat=-15.62158186029299), size=9), Delivery(id='62ad5b7f0d6c9e6905d387f48b3b0a6b', point=Point(lng=-47.70313801955018, lat=-15.622108309076024), size=3), Delivery(id='b32eaa4c4cb81b908f176d4ecc234a6d', point=Point(lng=-47.67692313890817, lat=-15.643914349833018), size=7), Delivery(id='2042cf5e9aa8eb8d5eb21e8f676276fc', point=Point(lng=-47.65123418176659, lat=-15.65293398867988), size=8), Delivery(id='e49455439e011d89bc68ad9175835cd3', point=Point(lng=-47.64485839823081, lat=-15.674032055167668), size=4), Delivery(id='65ec7bedae74bac46fa43626a0c4cafd', point=Point(lng=-47.6472675621646, lat=-15.674713959039739), size=7), Delivery(id='5ac9e1e9b207459ac5828f79cda48f09', point=Point(lng=-47.65093144661299, lat=-15.674532264155538), size=2), Delivery(id='18063ffe2b7d3157d

In [9]:
# Vamos conferir se a solução é factível, e qual a sua distância total
from loggibud.v1.distances import OSRMConfig
from loggibud.v1.eval.task1 import evaluate_solution


# Configuração com o servidor para os alunos
config = OSRMConfig(host="http://localhost:5000")

evaluate_solution(problem, solution, config=config)

1532.2272

In [10]:
# Aqui estão as rotas visualizadas em rua
from loggibud.v1.plotting.plot_solution import plot_cvrp_solution_routes


plot_cvrp_solution_routes(solution, config=config)

## `dev/pa-0/cvrp-0-pa-90.json`

In [ ]:
# Comece carregando a instância
from loggibud.v1.types import CVRPInstance


file_path = "../data/cvrp-instances-1.0/dev/pa-0/cvrp-0-pa-90.json"
problem = CVRPInstance.from_file(file_path)
problem

In [ ]:
print(f"A capacidade de cada veículo é: {problem.vehicle_capacity}")
print(f"A localização do ponto de partida é: {problem.origin}")
print(f"Esta instância possui um total de {len(problem.deliveries)} entregas")

In [ ]:
# Resolve o problema (pode levar um tempo)
solution = solve_loggibud_vrp(problem)
solution

In [ ]:
# Vamos conferir se a solução é factível, e qual a sua distância total
from loggibud.v1.distances import OSRMConfig
from loggibud.v1.eval.task1 import evaluate_solution


# Configuração com o servidor para os alunos
config = OSRMConfig(host="http://localhost:5000")

evaluate_solution(problem, solution, config=config)

In [ ]:
# Aqui estão as rotas visualizadas em rua
from loggibud.v1.plotting.plot_solution import plot_cvrp_solution_routes


plot_cvrp_solution_routes(solution, config=config)

## `dev/rj-0/cvrp-0-rj-90.json`

In [ ]:
# Comece carregando a instância
from loggibud.v1.types import CVRPInstance


file_path = "../data/cvrp-instances-1.0/dev/rj-0/cvrp-0-rj-90.json"
problem = CVRPInstance.from_file(file_path)
problem

In [ ]:
print(f"A capacidade de cada veículo é: {problem.vehicle_capacity}")
print(f"A localização do ponto de partida é: {problem.origin}")
print(f"Esta instância possui um total de {len(problem.deliveries)} entregas")

In [ ]:
# Resolve o problema (pode levar um tempo)
solution = solve_loggibud_vrp(problem)
solution

(No momento em que executei, o algoritmo levou mais de dez minutos - na verdade, quase meia hora - para finalizar)

In [ ]:
# Vamos conferir se a solução é factível, e qual a sua distância total
from loggibud.v1.distances import OSRMConfig
from loggibud.v1.eval.task1 import evaluate_solution


# Configuração com o servidor para os alunos
config = OSRMConfig(host="http://localhost:5000")

evaluate_solution(problem, solution, config=config)

In [ ]:
# Aqui estão as rotas visualizadas em rua
from loggibud.v1.plotting.plot_solution import plot_cvrp_solution_routes


plot_cvrp_solution_routes(solution, config=config)

# Exercício 2

Aqui está um código simples que resolve de uma vez as mesmas instâncias de antes com o LKH-3, já presente no repositório:

In [ ]:
from loggibud.v1.types import CVRPInstance
from loggibud.v1.distances import OSRMConfig
from loggibud.v1.eval.task1 import evaluate_solution
from loggibud.v1.baselines.task1 import lkh_3


file_paths = [
    "../data/cvrp-instances-1.0/dev/df-0/cvrp-0-df-90.json",
    "../data/cvrp-instances-1.0/dev/pa-0/cvrp-0-pa-90.json",
    "../data/cvrp-instances-1.0/dev/rj-0/cvrp-0-rj-90.json",
]
# Configuração com o servidor para os alunos
osrm_config = OSRMConfig(host="http://localhost:5000")

lkh_params = lkh_3.LKHParams(osrm_config=osrm_config)

for file_path in file_paths:
    problem = CVRPInstance.from_file(file_path)

    print(f"Instância: {problem.name}")
    print(f"A capacidade de cada veículo é: {problem.vehicle_capacity}")
    print(f"A localização do ponto de partida é: {problem.origin}")
    print(f"Esta instância possui um total de {len(problem.deliveries)} entregas")

    solution = lkh_3.solve(problem, params=lkh_params)

    total_distance = evaluate_solution(problem, solution, config=osrm_config)

    print(f"A solução tem distância total {total_distance} km\n\n")

Observe como as soluções do LKH-3 possuem distância total menor. Este algoritmo tende a ser mais eficiente que aqueles utilizados pelo OR-Tools, então pode ser que valha a pena [consultá-lo](http://webhotel4.ruc.dk/~keld/research/LKH-3/) caso tenha interesse na área.